<a href="https://colab.research.google.com/github/shashank3110/forecasting/blob/main/MIMO_Multivaraite_Forecast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget https://github.com/laiguokun/multivariate-time-series-data/raw/refs/heads/master/exchange_rate/exchange_rate.txt.gz



datset source: https://github.com/laiguokun/multivariate-time-series-data


In [3]:
!gunzip  exchange_rate.txt.gz

In [ ]:
!pip install skforecast==0.19.1

In [ ]:
!pip install ydata-profiling

In [6]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor

import tensorflow as tf
from tensorflow.keras import layers
from skforecast.deep_learning import ForecasterRnn

from skforecast.direct import ForecasterDirectMultiVariate
from skforecast.direct import ForecasterDirect
from sklearn.model_selection import train_test_split

In [7]:
file_path = '/content/exchange_rate.txt'


with open(file_path, 'r') as file:
    raw_df = pd.DataFrame([line.strip().split(',') for line in file])


In [8]:
raw_df.head(), raw_df.shape

(          0         1         2         3         4         5         6  \
 0  0.785500  1.611000  0.861698  0.634196  0.211242  0.006838  0.593000   
 1  0.781800  1.610000  0.861104  0.633513  0.211242  0.006863  0.594000   
 2  0.786700  1.629300  0.861030  0.648508  0.211242  0.006975  0.597300   
 3  0.786000  1.637000  0.862069  0.650618  0.211242  0.006953  0.597000   
 4  0.784900  1.653000  0.861995  0.656254  0.211242  0.006940  0.598500   
 
           7  
 0  0.525486  
 1  0.523972  
 2  0.526316  
 3  0.523834  
 4  0.527426  ,
 (7588, 8))

In [ ]:
raw_df.columns = [f'series_{idx+1}' for idx in raw_df.columns]
raw_df = raw_df.astype(float)

In [7]:
raw_df.head()

,series_1,series_2,series_3,series_4,series_5,series_6,series_7,series_8
0,0.785500,1.611000,0.861698,0.634196,0.211242,0.006838,0.593000,0.525486
1,0.781800,1.610000,0.861104,0.633513,0.211242,0.006863,0.594000,0.523972
2,0.786700,1.629300,0.861030,0.648508,0.211242,0.006975,0.597300,0.526316
3,0.786000,1.637000,0.862069,0.650618,0.211242,0.006953,0.597000,0.523834
4,0.784900,1.653000,0.861995,0.656254,0.211242,0.006940,0.598500,0.527426


In [20]:
raw_df.isna().sum()

,0
series_1,0
series_2,0
series_3,0
series_4,0
series_5,0
series_6,0
series_7,0
series_8,0


In [10]:
train_df = raw_df.iloc[:-90, :4]
test_df = raw_df.iloc[-90:, :4]

In [11]:
raw_df.shape, train_df.shape, test_df.shape

((7588, 8), (7498, 4), (90, 4))

### EDA: Data Profiling


In [ ]:
# Data Profiling
from ydata_profiling import ProfileReport

# Generate automated EDA report
profile = ProfileReport(raw_df, title="Multivariate Time Series EDA", tsmode=True)
profile.to_file("eda_report.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]


100%|██████████| 8/8 [00:11<00:00,  1.42s/it]


In [ ]:
# display data profiling results in notebook
profile.to_notebook_iframe()


In [22]:
raw_df['series_4'].isnull().sum()

np.int64(0)

# Modelling:


## A. Multivariate Direct forecasting: One model for every timestep in forecast horizon

- This is multioutput in series dimension i.e. not in time dimension: each model gives prediction for only one time step but it gives for all series.

- Number of models = Length of time horizon * Number of series

In [45]:
%%time


series = ['series_1', 'series_2', 'series_3', 'series_4']

predictions_direct = {}
for s_idx in series:
  # Setup: 30-day horizon with a 14-day lookback
  forecaster_direct = ForecasterDirectMultiVariate(
      regressor          = XGBRegressor(n_estimators=100, random_state=123),
      level              = s_idx,
      lags               = 14,
      steps              = 30,
      transformer_series = StandardScaler(), # Scales all 5 series independently
      n_jobs             = 'auto'            # Trains 30 models in parallel
  )

  # Training (Input: DataFrame with all 5 series)
  forecaster_direct.fit(series=train_df)

  # Predict (Output: In original scale)
  predictions_direct[s_idx] = forecaster_direct.predict(steps=30)
  print(f'{s_idx} trained and predicted')

<timed exec>:6: FutureWarning: The `regressor` argument is deprecated and will be removed in a future version. Please use `estimator` instead.


series_1 trained and predicted


<timed exec>:6: FutureWarning: The `regressor` argument is deprecated and will be removed in a future version. Please use `estimator` instead.


series_2 trained and predicted


<timed exec>:6: FutureWarning: The `regressor` argument is deprecated and will be removed in a future version. Please use `estimator` instead.


series_3 trained and predicted


<timed exec>:6: FutureWarning: The `regressor` argument is deprecated and will be removed in a future version. Please use `estimator` instead.


series_4 trained and predicted
CPU times: user 6min 27s, sys: 1.5 s, total: 6min 29s
Wall time: 4min 24s


## Results: Accuracy

In [49]:
direct_results1 = pd.concat([test_df.loc[7498:7527,'series_1'], predictions_direct['series_1']], axis=1)
direct_results1['accuracy_direct'] = 100.0 - (abs(direct_results1['series_1'] - direct_results1['pred'])*100.0)/direct_results1['series_1']



direct_results2 = pd.concat([test_df.loc[7498:7527,'series_2'], predictions_direct['series_2']], axis=1)
direct_results2['accuracy_direct'] = 100.0 - (abs(direct_results2['series_2'] - direct_results2['pred'])*100.0)/direct_results2['series_2']



direct_results3 = pd.concat([test_df.loc[7498:7527,'series_3'], predictions_direct['series_3']], axis=1)
direct_results3['accuracy_direct'] = 100.0 - (abs(direct_results3['series_3'] - direct_results3['pred'])*100.0)/direct_results3['series_3']



direct_results4 = pd.concat([test_df.loc[7498:7527,'series_4'], predictions_direct['series_4']], axis=1)
direct_results4['accuracy_direct'] = 100.0 - (abs(direct_results4['series_4'] - direct_results4['pred'])*100.0)/direct_results4['series_4']



In [50]:
direct_results1['accuracy_direct'].mean(), direct_results2['accuracy_direct'].mean(), direct_results3['accuracy_direct'].mean(), direct_results4['accuracy_direct'].mean()

(np.float64(99.35555575997333),
 np.float64(92.81257420219954),
 np.float64(97.28095398179958),
 np.float64(98.24969485759829))

## B. Multivariate MIMO Forecasting: DL

We can try same with XGB using multi_output_tree strategy: (see Section C)
- Setup MIMO XGBoost
```
mimo_regressor = XGBRegressor(
    tree_method    = "hist",
    multi_strategy = "multi_output_tree",
    n_estimators   = 100
)
```

In [21]:
# 1. Define a simple MIMO Neural Network architecture
def create_model(levels, lags, steps):
    input_layer = layers.Input(shape=(lags, levels), name='series_input') #layer name needs to be same for skforecast
    x = layers.LSTM(64, activation='relu')(input_layer)
    x = layers.Dense(64, activation='relu')(x)
    # Output must be (steps * levels) -> 30 * 5 = 150 units
    output_layer = layers.Dense(steps * levels, name='output_dense_td_layer')(x) #layer name needs to be same for skforecast
    # Reshape to (steps, levels) for skforecast compatibility
    output_layer = layers.Reshape((steps, levels))(output_layer)

    model = tf.keras.Model(inputs=input_layer, outputs=output_layer)
    model.compile(optimizer='adam', loss='mse')
    return model

# 2. Initialize the ForecasterRnn
# It handles the scaling and data formatting for the Neural Network
forecaster_rnn = ForecasterRnn(
    regressor      = create_model(levels=4, lags=14, steps=30),
    levels         = ['series_1', 'series_2', 'series_3', 'series_4'],
    lags           = 14,
    # steps          = 30,
    transformer_series = StandardScaler(),
    fit_kwargs     = {'epochs': 50, 'batch_size': 32, 'verbose': 0}
)

/tmp/ipython-input-3307453370.py:17: FutureWarning: The `regressor` argument is deprecated and will be removed in a future version. Please use `estimator` instead.
  forecaster_rnn = ForecasterRnn(
/usr/local/lib/python3.12/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'adam', because it has 16 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [22]:
%%time
# 3. Fit and Predict
forecaster_rnn.fit(series=train_df)
# Predictions are generated in one single 'shot'
predictions_rnn = forecaster_rnn.predict(steps=30)

CPU times: user 2min 21s, sys: 2.54 s, total: 2min 23s
Wall time: 1min 48s


In [39]:
results1 = pd.concat([test_df.loc[7498:7527,'series_1'],predictions_rnn[predictions_rnn['level']=='series_1']], axis=1)
results1['accuracy_rnn'] = 100.0 - (abs(results1['series_1'] - results1['pred'])*100.0)/results1['series_1']

results2 = pd.concat([test_df.loc[7498:7527,'series_2'],predictions_rnn[predictions_rnn['level']=='series_2']], axis=1)
results2['accuracy_rnn'] = 100.0 - (abs(results2['series_2'] - results2['pred'])*100.0)/results2['series_2']

results3 = pd.concat([test_df.loc[7498:7527,'series_3'],predictions_rnn[predictions_rnn['level']=='series_3']], axis=1)
results3['accuracy_rnn'] = 100.0 - (abs(results3['series_3'] - results3['pred'])*100.0)/results3['series_3']

results4 = pd.concat([test_df.loc[7498:7527,'series_4'],predictions_rnn[predictions_rnn['level']=='series_4']], axis=1)
results4['accuracy_rnn'] = 100.0 - (abs(results4['series_4'] - results4['pred'])*100.0)/results4['series_4']



## Results: Accuracy

In [43]:
results1['accuracy_rnn'].mean(), results2['accuracy_rnn'].mean(), results3['accuracy_rnn'].mean(), results4['accuracy_rnn'].mean()

(np.float64(99.27185548197392),
 np.float64(94.58379749352633),
 np.float64(99.44704777641591),
 np.float64(99.01009956646614))

In [52]:
help(ForecasterRnn) # also can add exog variable dataframe

Help on class ForecasterRnn in module skforecast.deep_learning._forecaster_rnn:

class ForecasterRnn(skforecast.base._forecaster_base.ForecasterBase)
 |  ForecasterRnn(levels: 'str | list[str]', lags: 'int | list[int] | np.ndarray[int] | range[int]', estimator: 'object' = None, transformer_series: 'object | dict[str, object] | None' = MinMaxScaler(), transformer_exog: 'object | None' = MinMaxScaler(), fit_kwargs: 'dict[str, object] | None' = {}, forecaster_id: 'str | int | None' = None, regressor: 'object' = None) -> 'None'
 |
 |  This class turns any estimator compatible with the Keras API into a
 |  Keras RNN multi-series multi-step forecaster. A unique model is created
 |  to forecast all time steps and series. Keras enables workflows on top of
 |  either JAX, TensorFlow, or PyTorch. See documentation for more details.
 |
 |  Parameters
 |  ----------
 |  estimator : estimator or pipeline compatible with the Keras API
 |      An instance of a estimator or pipeline compatible with th

## C. Multivariate MIMO Forecasting: XGB

- here we need to manually  initialize one model for every series as there is no skforecast wrapper for ML models for MIMO only for DL we have.

In [73]:
%%time

series = ['series_1', 'series_2', 'series_3', 'series_4']

predictions_mimo_xgb = {}
for s_idx in series:

  mimo_regressor = XGBRegressor(
      tree_method    = "hist",
      multi_strategy = "multi_output_tree",
      n_estimators   = 100
  )

  # Training (Input: DataFrame with all 5 series)
  X_train = train_df.drop(columns=[s_idx])
  mimo_regressor.fit(X=X_train , y=train_df[s_idx])

  X_test = test_df.drop(columns=[s_idx])

  # Predict (Output: In original scale)
  predictions_mimo_xgb[s_idx] = pd.DataFrame(mimo_regressor.predict(X_test.loc[7498:7527, :]), columns=['pred'], index=X_test.loc[7498:7527, :].index)
  print(f'{s_idx} trained and predicted')




series_1 trained and predicted
series_2 trained and predicted
series_3 trained and predicted
series_4 trained and predicted
CPU times: user 1.32 s, sys: 12.8 ms, total: 1.33 s
Wall time: 903 ms


In [59]:
X_test.shape, X_train.shape

((90, 3), (7498, 3))

In [76]:
mimo_xgb_results1 = pd.concat([test_df.loc[7498:7527,'series_1'], predictions_mimo_xgb['series_1']], axis=1)
mimo_xgb_results1['accuracy_mimo_xgb'] = 100.0 - (abs(mimo_xgb_results1['series_1'] - mimo_xgb_results1['pred'])*100.0)/mimo_xgb_results1['series_1']



mimo_xgb_results2 = pd.concat([test_df.loc[7498:7527,'series_2'], predictions_mimo_xgb['series_2']], axis=1)
mimo_xgb_results2['accuracy_mimo_xgb'] = 100.0 - (abs(mimo_xgb_results2['series_2'] - mimo_xgb_results2['pred'])*100.0)/mimo_xgb_results2['series_2']



mimo_xgb_results3 = pd.concat([test_df.loc[7498:7527,'series_3'], predictions_mimo_xgb['series_3']], axis=1)
mimo_xgb_results3['accuracy_mimo_xgb'] = 100.0 - (abs(mimo_xgb_results3['series_3'] - mimo_xgb_results3['pred'])*100.0)/mimo_xgb_results3['series_3']



mimo_xgb_results4 = pd.concat([test_df.loc[7498:7527,'series_4'], predictions_mimo_xgb['series_4']], axis=1)
mimo_xgb_results4['accuracy_mimo_xgb'] = 100.0 - (abs(mimo_xgb_results4['series_4'] - mimo_xgb_results4['pred'])*100.0)/mimo_xgb_results4['series_4']


## Results: Accuracy

In [77]:
mimo_xgb_results1['accuracy_mimo_xgb'].mean(), mimo_xgb_results2['accuracy_mimo_xgb'].mean(), mimo_xgb_results3['accuracy_mimo_xgb'].mean(), mimo_xgb_results4['accuracy_mimo_xgb'].mean()

(np.float64(97.96950321914166),
 np.float64(89.37202740089371),
 np.float64(98.52263768704988),
 np.float64(97.95665660271428))